# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

STEP 1: Keep only US patents with state

In [8]:
us_patents = patents.filter(
    (patents.COUNTRY == "US") & (patents.POSTATE.isNotNull())
).select(
    patents.PATENT.alias("PATENT_ID"),
    patents.POSTATE.alias("STATE")
)


STEP 2: Join citations with cited patent state

In [9]:
cited = citations.join(
    us_patents,
    citations.CITED == us_patents.PATENT_ID,
    how="left"
).select(
    citations.CITING.alias("CITING"),
    citations.CITED.alias("CITED"),
    us_patents.STATE.alias("CITED_STATE")
)


STEP 3: Join again to get citing patent state

In [10]:
joined = cited.join(
    us_patents,
    cited.CITING == us_patents.PATENT_ID,
    how="left"
).select(
    cited.CITING,
    cited.CITED,
    cited.CITED_STATE,
    us_patents.STATE.alias("CITING_STATE")
)


STEP 4: Keep only same-state citations

In [11]:
same_state = joined.filter(
    (joined.CITED_STATE.isNotNull()) &
    (joined.CITED_STATE == joined.CITING_STATE)
)


STEP 5: Count same-state citations per patent

In [12]:
same_state_counts = same_state.groupBy("CITING") \
    .count() \
    .withColumnRenamed("count", "SAME_STATE")


STEP 6: Join counts back to patent table

In [13]:
final_df = patents.join(
    same_state_counts,
    patents.PATENT == same_state_counts.CITING,
    how="left"
).fillna(0, subset=["SAME_STATE"])


STEP 7: Show top 10

In [14]:
final_df.select("PATENT", "SAME_STATE") \
    .orderBy("SAME_STATE", ascending=False) \
    .show(10)


+-------+----------+
| PATENT|SAME_STATE|
+-------+----------+
|5959466|       125|
|5983822|       103|
|6008204|       100|
|5952345|        98|
|5958954|        96|
|5998655|        96|
|5936426|        94|
|5739256|        90|
|5913855|        90|
|5925042|        90|
+-------+----------+
only showing top 10 rows

